# SW중심대학 디지털 경진대회_SW와 생성AI의 만남 : AI부문
 - 이 AI 경진대회에서는 5초 분량의 오디오 샘플에서 진짜 사람 목소리와 AI가 생성한 가짜 목소리를 정확하게 구분할 수 있는 모델을 개발하는 것이 목표입니다.
 - 이 작업은 보안, 사기 감지 및 오디오 처리 기술 향상 등 다양한 분야에서 매우 중요합니다.

## Imports
모델 학습 및 추론에 사용할 라이브러리들을 불러옵니다.

In [2]:
import os
import random

import torch
import torchaudio

import librosa
import numpy as np
import pandas as pd

from torch import nn
import torch.nn.functional as F
import torchaudio.transforms as T
from torch.utils.data import Dataset, DataLoader

from tqdm.notebook import tqdm

### Check GPU Availability

In [3]:
!nvidia-smi

Mon Jul 15 16:34:21 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.29.05    Driver Version: 495.29.05    CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  On   | 00000000:04:00.0 Off |                    0 |
| N/A   41C    P0    34W / 250W |   1928MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla P100-PCIE...  On   | 00000000:06:00.0 Off |                    0 |
| N/A   

In [4]:
# Set CUDA Device Number 0~7
DEVICE_NUM = 0

device = torch.device("cpu")
if torch.cuda.is_available():
    torch.cuda.set_device(DEVICE_NUM)
    device = torch.device("cuda")
print("INFO: Using device -", device)

INFO: Using device - cuda


## Config
- 딥러닝 모델을 학습하기 전에 설정해야하는 다양한 매개변수를 정의하는 설정 클래스입니다.
- 클래스를 사용하여 학습에 필요한 설정 값을 미리 지정합니다.

##### 오디오 신호
- 우리가 듣는 소리는 공기의 압력 변화로, 이것을 디지털 신호로 변환한 것이 오디오 신호입니다.
- 이 신호는 시간에 따라 변하는 진폭 값을 가지고 있습니다.

In [5]:
class Config:
    """ Configuration Class """
    SEED = 20240719  # 재현성을 위해 랜덤 시드 고정
    NB_NAME = "wespeaker_augmented"  # ipython 노트북 이름 지정
    
    """ SR(Sample Rate)
    - 오디오 데이터의 샘플링 레이트를 설정합니다.
    - 높은 샘플링 레이트는 더 높은 주파수의 소리를 캡처할 수 있지만, 처리에 더 많은 계산 자원이 필요합니다.
    - 오디오 데이터의 초당 샘플 수를 정의합니다.
    """
    SR = 32000

    """ N_MFCC(Number of MFCCs)
    - N_MFCC는 멜 주파수 켑스트럼 계수(MFCCs)의 개수를 의미합니다.
    - MFCC는 오디오 신호의 주파수 특성을 인간의 청각 특성에 맞게 변환한 것으로, 음성 인식 등의 분야에서 많이 사용됩니다.
    """ 
    N_MFCC = 13

    """ ROOT_FOLDER
    - 데이터셋의 루트 폴더 경로를 설정합니다.
    """
    ROOT_FOLDER = os.path.join(".", "data")
    
    """ BATCH_SIZE
    - 학습 시 한 번에 처리할 데이터 샘플의 수를 정의합니다
    - 큰 배치 크기는 메모리 사용량을 증가시키지만, 학습 속도를 높입니다.
    """
    BATCH_SIZE = 100
    
    """ N_EPOCHS
    - 전체 데이터셋을 학습할 횟수를 정의합니다.
    - 에폭 수가 너무 적으면 과소적합이 발생할 수 있고, 너무 많으면 과적합이 발생할 수 있습니다.
    """
    N_EPOCHS = 20
    
    """ LR (Learning Rate)
    - 모델의 가중치를 업데이트할 때 사용되는 학습 속도를 정의합니다.
    - 학습률이 너무 크면 학습이 불안정해질 수 있고, 너무 작으면 학습 속도가 느려집니다.
    """
    LR = 3e-4

In [6]:
def seed_everything(seed):
    """ Fixed RandomSeed
    아래의 코드는 머신러닝이나 딥러닝 모델을 훈련할 때, 결과의 재현성을 보장하기 위해 사용되는 함수입니다.
    이 함수는 다양한 랜덤 시드를 고정하여, 실행할 때마다 동일한 결과를 얻기 위해 사용됩니다.
    """
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(Config.SEED)  # Seed 고정

## Dataset

In [7]:
from torchvision.datasets import utils
from sklearn.model_selection import train_test_split as split

utils.tqdm = tqdm


class VoiceDataset(Dataset):
    download_url = "https://drive.usercontent.google.com/download?id=1hi1dibkHyFbaxAteLlZJw6r3g9ddd4Lf&export=download&authuser=0&confirm=t&uuid=c40c278b-d74b-4b75-bc79-09e8a3ccffa4&at=APZUnTUvIVFVM9gjGNUCmDb4YZCy%3A1719807236671"

    @classmethod
    def download(cls, root='./data', filename="download.zip", md5=None):
        cls.download_root = root
        filepath = os.path.join(root, filename)
        if not os.path.exists(filepath):
            utils.download_and_extract_archive(cls.download_url, root, root, filename, md5)
            print("Extraction completed.")
        else:
            print(f"File already exists in {filepath}")

    @property
    def get_dataset_path(self):
        filename = "train.csv" if self.is_train else "test.csv"
        if self.custom_csv:
            filename = self.custom_csv + ".csv"
        return os.path.join(self.download_root, filename)

    @property
    def submission_form_path(cls):
        return os.path.join(cls.download_root, "sample_submission.csv")

    def __init__(self, root="./data", train=True, split_ratio=1, transform=None, custom_csv=None):
        """
        Voice Dataset for Contrastive Learning
        
        :param root: The path to the data directory
        :param train: is train or test
        :param split_ratio: split ratio for train(can be 0.5 or above) and valid(can be lower than 0.5) set
        :param transform: data transformer
        :param target_transform: label transformer
        """
        super().__init__()
        self.download(root)
        self.download_root = root
        self.is_train = train
        self.custom_csv = custom_csv
        self.dataset_name = ("train" if train else "test") if not custom_csv else custom_csv

        raw_data = self._load_data(self.get_dataset_path, split_ratio if split_ratio >= 0.5 else 1-split_ratio)
        if not self.is_train or split_ratio >= 0.5:
            self.raw_data, _ = raw_data
        else:
            _, self.raw_data = raw_data
            if "train" not in self.dataset_name:
                print(f"Warning: The name of dataset should start with 'train' for training set. (current - {self.dataset_name})")
            self.dataset_name = self.dataset_name.replace("train", "valid")

        self.data = self.raw_data['path'].tolist()

        if 'label' in self.raw_data.columns:
            self.label = [(0, 1) if lb == 'real' else (1, 0) for lb in self.raw_data['label'].tolist()]
        else:
            if 'real' in self.raw_data.columns and 'fake' in self.raw_data.columns:
                f_label = self.raw_data['fake'].tolist()
                r_label = self.raw_data['real'].tolist()
                self.label = list(zip(f_label, r_label))
            else:
                self.label = None

        self.transforms(transform)

    @staticmethod
    def _load_data(dataset_path, split_ratio=1):
        random_state = 1  # fixed random_state

        df = pd.read_csv(dataset_path)

        if split_ratio == 1 or split_ratio == 0:
            return (df, None) if split_ratio == 1 else (None, df)

        if 'label' in df.columns:
            df1, df2, _, _ = split(df, df['label'], test_size=1-split_ratio, random_state=random_state)
        else:
            df1, df2 = split(df, test_size=1-split_ratio, random_state=random_state)
        return df1, df2

    def transforms(self, transform=None):
        if transform is not None:
            if not isinstance(transform, list) and not isinstance(transform, tuple):
                transform = [transform]
            for t in transform:
                self.data, self.label = t(self.data, self.label)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        if self.label is not None:
            return self.data[index], self.label[index]
        return self.data[index]

In [8]:
split_ratio = 0.8

train_dataset = VoiceDataset(root=Config.ROOT_FOLDER, train=True, split_ratio=split_ratio)
train_augmented = VoiceDataset(root=Config.ROOT_FOLDER, train=True, split_ratio=split_ratio, custom_csv="train_augmented")
valid_dataset = VoiceDataset(root=Config.ROOT_FOLDER, train=True, split_ratio=1-split_ratio)
valid_augmented = VoiceDataset(root=Config.ROOT_FOLDER, train=True, split_ratio=1-split_ratio, custom_csv="train_augmented")
unlabeled_dataset = VoiceDataset(root=Config.ROOT_FOLDER, train=False, custom_csv="unlabeled_data")
test_dataset = VoiceDataset(root=Config.ROOT_FOLDER, train=False)

print(f"Loaded Dataset - train({len(train_dataset)}), valid({len(valid_dataset)}), unlabeled({len(unlabeled_dataset)}) test({len(test_dataset)})")
print("Query Dataset for checking:", train_dataset[0])
train_dataset.raw_data

File already exists in ./data/download.zip
File already exists in ./data/download.zip
File already exists in ./data/download.zip
Loaded Dataset - train(44350), valid(11088), test(50000)
Query Dataset for checking: ('./train/NQJUDUMG.ogg', (1, 0))


,id,path,label
19535,NQJUDUMG,./train/NQJUDUMG.ogg,fake
37414,SGACBBDI,./train/SGACBBDI.ogg,fake
40645,SIBSFMAP,./train/SIBSFMAP.ogg,fake
16487,LLBQPFAD,./train/LLBQPFAD.ogg,real
954,ZWYRTAOF,./train/ZWYRTAOF.ogg,real
...,...,...,...
50057,BDFFJCBX,./train/BDFFJCBX.ogg,fake
32511,NEFSVUCS,./train/NEFSVUCS.ogg,real
5192,MJFGSHIR,./train/MJFGSHIR.ogg,fake
12172,USIDOXOR,./train/USIDOXOR.ogg,real


#### Data Transformation

In [10]:
audio_cache_dir = "audio_cache"  # audio cache directory

if not os.path.isdir(audio_cache_dir):
    os.mkdir(audio_cache_dir)
if not os.path.isdir(os.path.join(audio_cache_dir, Config.NB_NAME)):
    os.mkdir(os.path.join(audio_cache_dir, Config.NB_NAME))

In [11]:
from librosa.feature import mfcc

def to_embedding(dataset, label, sample_rate=Config.SR):
    convert_path = lambda path: os.path.join(Config.ROOT_FOLDER, *path.replace("./", "").split("/"))

    features = []

    for d in tqdm(dataset):
        y, sr = librosa.load(convert_path(d), sr=sample_rate)
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=sample_rate)
        features.append(np.mean(mfcc.T, axis=0))

    return features, label

In [12]:
to_tensor = lambda d, l: (d, list(map(torch.tensor, l)))  # label to tensor

train_dataset.transforms(transform=[to_embedding, to_tensor])
train_augmented.transforms(transform=[to_embedding, to_tensor])
valid_dataset.transforms(transform=[to_embedding, to_tensor])
valid_augmented.transforms(transform=[to_embedding, to_tensor])
test_dataset.transforms(transform=[to_embedding])

INFO: Pretrained Voice Embedding loaded. - id(140145537301056), type(<class 'list'>)
INFO: Pretrained Voice Embedding loaded. - id(140145537301312), type(<class 'list'>)
INFO: Pretrained Voice Embedding loaded. - id(140144730675584), type(<class 'list'>)


In [13]:
for (data, label), i in zip(train_dataset, range(5)):
    print(f"Dataset {i}: {label}", data)

Dataset 0: tensor([1, 0]) tensor([-0.2622, -0.1319, -0.0539,  0.0124,  0.1426,  0.1331,  0.0709,  0.0193,
         0.0863, -0.2342,  0.0471,  0.2622, -0.0538,  0.0060,  0.0194, -0.0204,
        -0.1776,  0.0919,  0.1826,  0.0205, -0.0145, -0.0908,  0.0730,  0.1513,
         0.0507,  0.1620,  0.0702, -0.1969, -0.0679, -0.1909, -0.0514, -0.0315,
        -0.1292, -0.1566,  0.0784,  0.2276,  0.0624,  0.0883, -0.0481,  0.2119,
         0.1538, -0.0134,  0.1566,  0.0766, -0.0213, -0.1333, -0.0209, -0.0596,
         0.0964,  0.1612, -0.1159, -0.2910, -0.0109,  0.1471, -0.2822, -0.0772,
         0.1346, -0.0208,  0.0997,  0.2245,  0.0230,  0.0578, -0.0675,  0.0491,
        -0.1563,  0.0842, -0.1240, -0.0403, -0.0722,  0.0857, -0.1086, -0.0910,
        -0.1778,  0.1218, -0.0394,  0.0036,  0.1889,  0.4586, -0.2083,  0.1329,
        -0.2099,  0.0219,  0.1260,  0.0328,  0.1316,  0.0938, -0.0038,  0.0322,
        -0.1676,  0.0230, -0.0543, -0.0173,  0.0925,  0.0832,  0.0672, -0.0914,
         0.004

In [14]:
for (data, label), i in zip(valid_dataset, range(5)):
    print(f"Dataset {i}: {label}", data)

Dataset 0: tensor([1, 0]) tensor([-0.0336,  0.0182,  0.1605, -0.0633, -0.2657, -0.0786,  0.2894, -0.0773,
        -0.0861,  0.1535,  0.2038, -0.1084, -0.0763,  0.0494,  0.2047, -0.0693,
         0.2637,  0.0098,  0.0902,  0.0144,  0.0416,  0.0160,  0.1329,  0.1867,
        -0.0150,  0.0670,  0.0060, -0.2174,  0.0260, -0.1373,  0.1242,  0.1294,
        -0.0187,  0.1399,  0.0461,  0.1157,  0.1255,  0.0183, -0.0848,  0.0068,
        -0.0793,  0.0474, -0.0455, -0.1096, -0.0644,  0.0303, -0.0370, -0.1810,
        -0.1520,  0.0405, -0.0606, -0.0048,  0.0287, -0.0114, -0.0170, -0.1477,
         0.0111, -0.0047,  0.1893, -0.2278, -0.1438,  0.0524,  0.1691, -0.2530,
         0.0281, -0.0036, -0.0240, -0.1658, -0.1500,  0.1094,  0.2081, -0.0912,
        -0.1184,  0.0304,  0.0684,  0.0259,  0.1747,  0.2028,  0.0706, -0.1569,
        -0.0617,  0.0943,  0.1673,  0.1042,  0.0533, -0.1656, -0.0861,  0.0449,
        -0.1007,  0.0041, -0.2378, -0.0718,  0.2067, -0.0689,  0.2285,  0.1158,
        -0.138

In [15]:
for dataset, i in zip(test_dataset, range(5)):
    print(f"Dataset {i}:", dataset)

Dataset 0: tensor([-0.0440,  0.0194, -0.0009, -0.0082, -0.0174, -0.1296, -0.0795, -0.0763,
         0.1436,  0.0484, -0.0266, -0.0678,  0.0314,  0.0616, -0.0423, -0.0676,
        -0.1374,  0.0819,  0.0537,  0.0678, -0.0810, -0.1357,  0.0359,  0.0167,
         0.0724,  0.0702,  0.1178,  0.1597,  0.0781,  0.0037,  0.1121,  0.0362,
        -0.0505, -0.0568,  0.0253,  0.0175, -0.0293,  0.0337, -0.1793, -0.0305,
        -0.0153, -0.0454, -0.0522,  0.2037,  0.1051, -0.0733, -0.0042,  0.0441,
        -0.0011,  0.1169,  0.0618,  0.0202, -0.0422,  0.0272, -0.0136, -0.1012,
        -0.0886,  0.0227,  0.0316, -0.0563, -0.0129,  0.0821, -0.0032, -0.0357,
         0.0695,  0.0099,  0.0551, -0.0480, -0.0132, -0.0537,  0.0250,  0.2138,
        -0.0260,  0.0234, -0.1294, -0.1327, -0.1087,  0.0559, -0.1272, -0.0230,
        -0.0462,  0.0715,  0.0991,  0.1146, -0.1166, -0.2101, -0.0384, -0.0236,
         0.0557,  0.0326, -0.1009, -0.0410,  0.0916, -0.1731,  0.1195, -0.1009,
        -0.1256,  0.0016,  0.

## DataLoader
    - DataLoader는 구축된 데이터셋에서 배치크기(batch_size)에 맞게 데이터를 추출하고, 필요에 따라 섞거나(shuffle=True) 순서대로 반환(shuffle=False)하는 역할을 합니다.
    - 훈련 데이터(train_loader)는 일반적으로 섞어서 모델이 데이터에 덜 편향되게 학습하도록하며,
      검증 데이터(val_loader)는 모델 성능 평가를 위해 순서대로 사용하고,
      테스트 데이터(test_loader)는 최종적인 추론을 위해 사용합니다.

    이렇게 DataLoader를 사용함으로써, 효율적인 데이터 처리와 모델 학습 및 평가가 가능해집니다.

In [16]:
BATCH_SIZE = Config.BATCH_SIZE

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
train_aug_loader = DataLoader(train_augmented, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
valid_aug_loader = DataLoader(valid_augmented, batch_size=BATCH_SIZE, shuffle=False)
unlabeled_loader = DataLoader(unlabeled_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

## Define Model

In [17]:
class VoiceEncoder(nn.Module):
    """ Voice Encoder Model """
    
    def __init__(self, embedding_dim, hidden_dim, output_size):
        super().__init__()
        self.fc1 = nn.Linear(embedding_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        h1 = self.relu(self.fc1(x))
        out = self.fc2(h1)
        return out

In [18]:
class VoiceDiscriminator(nn.Module):
    """ Voice Discriminator Model """
    
    def __init__(self, embedding_size, hidden_size, latent_size):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(embedding_size, hidden_size),
            nn.Tanh(),
            nn.Dropout(0.5),
            nn.Linear(hidden_size, hidden_size//2),
            nn.Tanh(),
            nn.Dropout(0.5),
            nn.Linear(hidden_size//2, latent_size)
        )
        self.classifier = nn.Linear(latent_size, 2)

    def forward(self, x):
        encoded = self.encoder(x)
        out = self.classifier(encoded)
        return F.sigmoid(out)

In [21]:
# 모델 파라미터 지정
model_params = dict(
    embedding_size=Config.N_MFCC,
    hidden_size=1024,
    latent_size=128
)
model_params

{'embedding_size': 256, 'hidden_size': 1024, 'latent_size': 128}

In [22]:
# 모델 생성
model = VoiceDiscriminator(**model_params)
model.to(device)

VoiceDiscriminator(
  (encoder): Sequential(
    (0): Linear(in_features=256, out_features=1024, bias=True)
    (1): Tanh()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=1024, out_features=512, bias=True)
    (4): Tanh()
    (5): Dropout(p=0.5, inplace=False)
    (6): Linear(in_features=512, out_features=128, bias=True)
  )
  (classifier): Linear(in_features=128, out_features=2, bias=True)
)

In [23]:
# BinaryCrossEntropy
criterion = nn.BCELoss().to(device)

# Adam optimizer
optimizer = torch.optim.Adam(params=model.parameters(), lr=Config.LR)

## Train & Validation

In [24]:
from sklearn.metrics import roc_auc_score

def multi_label_auc(y_true, y_scores):
    auc_scores = []
    for i in range(y_true.shape[1]):
        auc = roc_auc_score(y_true[:, i], y_scores[:, i])
        auc_scores.append(auc)
    mean_auc_score = np.mean(auc_scores)
    return mean_auc_score

In [25]:
# Set Epoch Count
num_epochs = Config.N_EPOCHS
log_interval = 5

In [26]:
last_val_score = 0
train_len, valid_len = map(len, (train_loader, valid_loader))

epochs = tqdm(range(1, num_epochs+1), desc="Running Epochs")
with tqdm(total=train_len, desc="Training") as train_progress, tqdm(total=valid_len, desc="Validation") as valid_progress:
    for epoch in epochs:
        train_progress.reset(total=train_len)
        valid_progress.reset(total=valid_len)

        # Train
        model.train()
        for i, inputs in enumerate(train_loader):
            optimizer.zero_grad()

            features, labels = (data.float().to(device) for data in inputs)
            outputs = model(features)

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_progress.update(1)
            print(f"\rEpoch [{epoch}/{num_epochs}], Step [{i+1}/{train_len}], Loss: {loss.item():.6f}", end="")

        val_loss, val_labels, val_outputs = 0, [], []

        # Validation
        model.eval()
        with torch.no_grad():
            for inputs in valid_loader:
                features, labels = (data.float().to(device) for data in inputs)
                predicted = model(features)

                val_loss += criterion(predicted, labels).item() / valid_len
                val_labels.append(labels.cpu().numpy())
                val_outputs.append(predicted.cpu().numpy())

                valid_progress.update(1)

        last_val_score = multi_label_auc(np.concatenate(val_labels, axis=0), np.concatenate(val_outputs, axis=0))  # Calculate AUC score

        print(f"\rEpoch [{epoch}/{num_epochs}], Step [{train_len}/{train_len}], Loss: {loss.item():.6f}, "
            + f"Valid Acc: {last_val_score:.6%}, Valid Loss: {val_loss:.6f}", end="\n" if epoch % log_interval == 0 or epoch == num_epochs else "")

Running Epochs:   0%|          | 0/7 [00:00<?, ?it/s]

Training:   0%|          | 0/444 [00:00<?, ?it/s]

Validation:   0%|          | 0/111 [00:00<?, ?it/s]

Epoch [5/7], Step [444/444], Loss: 0.108288, Valid Acc: 99.778335%, Valid Loss: 0.060704
Epoch [7/7], Step [444/444], Loss: 0.067874, Valid Acc: 99.821082%, Valid Loss: 0.051988


In [ ]:
# Set Epoch Count
num_epochs = 200
log_interval = 5

In [ ]:
last_val_score = 0
train_len, valid_len = map(len, (train_aug_loader, valid_aug_loader))

epochs = tqdm(range(1, num_epochs+1), desc="Running Epochs")
with tqdm(total=train_len, desc="Training") as train_progress, tqdm(total=valid_len, desc="Validation") as valid_progress:
    for epoch in epochs:
        train_progress.reset(total=train_len)
        valid_progress.reset(total=valid_len)

        # Train
        model.train()
        for i, inputs in enumerate(train_aug_loader):
            optimizer.zero_grad()

            features, labels = (data.float().to(device) for data in inputs)
            outputs = model(features)

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_progress.update(1)
            print(f"\rEpoch [{epoch}/{num_epochs}], Step [{i+1}/{train_len}], Loss: {loss.item():.6f}", end="")

        val_loss, val_labels, val_outputs = 0, [], []

        # Validation
        model.eval()
        with torch.no_grad():
            for inputs in valid_aug_loader:
                features, labels = (data.float().to(device) for data in inputs)
                predicted = model(features)

                val_loss += criterion(predicted, labels).item() / valid_len
                val_labels.append(labels.cpu().numpy())
                val_outputs.append(predicted.cpu().numpy())

                valid_progress.update(1)

        last_val_score = multi_label_auc(np.concatenate(val_labels, axis=0), np.concatenate(val_outputs, axis=0))  # Calculate AUC score

        print(f"\rEpoch [{epoch}/{num_epochs}], Step [{train_len}/{train_len}], Loss: {loss.item():.6f}, "
            + f"Valid Acc: {last_val_score:.6%}, Valid Loss: {val_loss:.6f}", end="\n" if epoch % log_interval == 0 or epoch == num_epochs else "")

In [ ]:
if not os.path.isdir(os.path.join(".", "models")):
    os.mkdir(os.path.join(".", "models"))

# Model Save
save_path = os.path.join(".", "models", f"{Config.NB_NAME}_acc_{last_val_score*100:.6f}.pt")
torch.save(model.state_dict(), save_path)
print(f"Model saved to {save_path}")

### Inference
테스트 데이터셋에 대한 추론은 다음 순서로 진행됩니다.

1. 모델 및 디바이스 설정
    - 모델을 주어진 device(GPU 또는 CPU)로 이동시키고, 평가모드로 전환합니다.
2. 예측 수행
    - 예측 결과를 저장한 빈 리스트를 초기화하고 test_loader에서 배치별로 데이터를 불러와 예측을 수행합니다.
    - 각 배치에 대해 스펙트로그램 데이터를 device로 이동시킵니다.
    - 모델 예측 확률(probs)을 계산합니다.
    - 예측 확률을 predictions리스트에 추가합니다.

In [ ]:
predicted_labels = []

model.to(device)
model.eval()
with torch.no_grad():
    for features in tqdm(test_loader):
        probs = model(features.to(device))
        probs = probs.cpu().detach().numpy()
        predicted_labels += probs.tolist()

### Submission
추론 결과를 제출 양식에 덮어 씌워 CSV 파일로 생성하는 과정은 다음과 같습니다.

1. 제출 양식 로드
    - pd.read_csv('./sample_submission.csv')를 사용하여 제출을 위한 샘플 형식 파일을 로드합니다.
    - 이 파일은 일반적으로 각 테스트 샘플에 대한 ID와 예측해야 하는 필드가 포함된 템플릿 형태를 가지고 있습니다.
2. 예측 결과 할당
    - submit.iloc[:,1:] = preds 추론함수(inference)에서 반환된 예측결과(preds)를 샘플 제출 파일에 2번째 열부터 할당합니다.
3. 제출 파일 저장
    - 수정된 제출 파일을 baseline_submit 이란 이름의 CSV 파일로 저장합니다.
    - index=False는 파일 저장시 추가적인 index가 발생하지 않도록 설정하여, 제작한 제출 파일과 동일한 형태의 파일을 저장합니다.

In [ ]:
submit = pd.read_csv(test_dataset.submission_form_path)
submit.iloc[:, 1:] = predicted_labels
submit.head()

In [ ]:
submission_dir = "submissions"
if not os.path.isdir(submission_dir):
    os.mkdir(submission_dir)

submit_file_path = os.path.join(".", submission_dir, f"{Config.NB_NAME}_acc_{last_val_score*100:.6f}_submit.csv")
submit.to_csv(submit_file_path, index=False)
print("File saved to", submit_file_path)